<a href="https://colab.research.google.com/github/skj092/Image-Classification-with-CIFAR-100/blob/main/exp3/ResnetPretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from keras.datasets import cifar100
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
num_classes = 100
nb_epochs = 10

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

#Pre-process the data
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

datagen = ImageDataGenerator()
datagen.fit(x_train)

In [ ]:
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [ ]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

model = Sequential()
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#from keras.callbacks import ModelCheckpoint, LearningRateScheduler

#checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
#                               verbose=1, save_best_only=True)
t=time.time()
#x_train.shape[0] // 64
historytemp = model.fit(datagen.flow(x_train, y_train,
                                  batch_size=100),
                                  steps_per_epoch=350,
                                  epochs=5,
                                  validation_data=(x_test, y_test))
print('Training time: %s' % (t - time.time()))

Epoch 1/5
350/350 [==============================] - 371s 1s/step - loss: 2.0934 - accuracy: 0.4751 - val_loss: 1.2392 - val_accuracy: 0.6458
Epoch 2/5
350/350 [==============================] - 365s 1s/step - loss: 1.0798 - accuracy: 0.6879 - val_loss: 0.9739 - val_accuracy: 0.7144
Epoch 3/5
350/350 [==============================] - 365s 1s/step - loss: 0.8638 - accuracy: 0.7444 - val_loss: 0.8573 - val_accuracy: 0.7420
Epoch 4/5
350/350 [==============================] - 365s 1s/step - loss: 0.7199 - accuracy: 0.7823 - val_loss: 0.8033 - val_accuracy: 0.7586
Epoch 5/5
350/350 [==============================] - 365s 1s/step - loss: 0.6296 - accuracy: 0.8088 - val_loss: 0.7789 - val_accuracy: 0.7681
Training time: -1865.4502747058868
